<a href="https://colab.research.google.com/github/JeyScientist/Artificial-Intelligence/blob/main/Copy_of_object2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
  Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires scipy>=1.10.0, but you have scipy 1.9.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.2 which is incompatible.
cvxpy 1.6.4 requires scipy>=1.11.0, but you have scipy 1.9.3 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.1.3 which is incompatible.
jax 0.5.2 req

In [ ]:
!pip install deep_sort_realtime

ERROR: Operation cancelled by user
^C


In [ ]:
!pip install DeepSORT

In [ ]:
!pip install --upgrade scipy tensorflow numba ultralytics
!pip install numpy==1.23.5
!pip uninstall numpy -y


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.6 MB/s eta 0:00:00
  Using cached numpy-2.1.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: llvmlite
    Foun

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DaBaby - _BOOK IT_ (Official Music Video)(360P).mp4 to DaBaby - _BOOK IT_ (Official Music Video)(360P) (1).mp4


In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort  # Using the more maintained version
from collections import defaultdict, deque
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import pandas as pd
import time

# First install compatible versions (run this in Colab)
!pip install numpy==1.23.5 ultralytics deep-sort-realtime opencv-python-headless

# ✅ Initialize YOLOv8
model = YOLO("yolov8n.pt")  # Using nano model for faster performance
model.fuse()

# ✅ Initialize DeepSORT
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    max_cosine_distance=0.3,
    nn_budget=None,
    override_track_class=None,
    embedder="mobilenet",
    half=True,
    bgr=True,
    embedder_gpu=True,
    embedder_model_name=None,
    embedder_wts=None,
    polygon=False,
    today=None
)

# 📐 Define Multi-Zone ROI (multiple polygon zones)
roi_zones = [
    np.array([(50, 100), (600, 80), (620, 400), (70, 420)], dtype=np.int32),  # Zone 1
    np.array([(700, 100), (1200, 80), (1220, 400), (700, 420)], dtype=np.int32),  # Zone 2
]

# 🔔 Alert Settings
alert_objects = ["person", "car"]

# 🗂️ Stats Containers
total_counts = defaultdict(int)
object_stats = []
track_memory = {}
trail_memory = defaultdict(lambda: deque(maxlen=30))
counted_ids = set()
speeds = defaultdict(float)

# 🖼️ Heatmap Parameters
heatmap = None

def point_inside_polygon(point, polygon):
    return cv2.pointPolygonTest(polygon, point, False) >= 0

def draw_polygon_roi(frame, polygon):
    cv2.polylines(frame, [polygon], isClosed=True, color=(255, 0, 0), thickness=2)
    cv2.putText(frame, "ROI", (polygon[0][0] + 10, polygon[0][1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

def draw_heatmap(frame):
    if heatmap is not None:
        heatmap_resized = cv2.resize(heatmap, (frame.shape[1], frame.shape[0]))
        heatmap_resized = cv2.applyColorMap(np.uint8(heatmap_resized), cv2.COLORMAP_JET)
        frame = cv2.addWeighted(frame, 1.0, heatmap_resized, 0.5, 0)
    return frame

def process_video(video_path, output_path="output_tracked.mp4", display=True, count_interval_seconds=60):
    global heatmap

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_idx = 0
    prev_centroids = {}

    with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc="Processing") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_idx += 1
            pbar.update(1)

            # Draw all ROI zones
            for zone in roi_zones:
                draw_polygon_roi(frame, zone)

            # Run YOLO detection
            results = model(frame, conf=0.6, iou=0.5, verbose=False)[0]

            # Prepare detections for DeepSORT
            detections = []
            for box in results.boxes:
                xyxy = box.xyxy[0].cpu().numpy()
                conf = float(box.conf)
                cls_id = int(box.cls)
                class_name = results.names[cls_id]

                # Check if center is in any ROI
                cx, cy = int((xyxy[0] + xyxy[2]) / 2), int((xyxy[1] + xyxy[3]) / 2)
                in_roi = any(point_inside_polygon((cx, cy), zone) for zone in roi_zones)

                if in_roi:
                    detections.append((xyxy, conf, class_name))

            # Update tracker
            if detections:
                bboxes = np.array([d[0] for d in detections])
                confidences = np.array([d[1] for d in detections])
                class_names = [d[2] for d in detections]

                tracks = tracker.update_tracks(bboxes, confidences, class_names, frame)

                for track in tracks:
                    if not track.is_confirmed():
                        continue

                    track_id = track.track_id
                    ltrb = track.to_ltrb()
                    x1, y1, x2, y2 = map(int, ltrb)
                    cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
                    class_name = track.get_class()
                    color = (0, 255, 0)

                    # Track duration
                    if track_id not in track_memory:
                        track_memory[track_id] = time.time()

                    # Count logic
                    duration = time.time() - track_memory[track_id]
                    if track_id not in counted_ids and duration >= count_interval_seconds:
                        total_counts[class_name] += 1
                        counted_ids.add(track_id)

                    # Speed calculation
                    if track_id in prev_centroids:
                        prev_x, prev_y = prev_centroids[track_id]
                        speed = np.sqrt((cx - prev_x)**2 + (cy - prev_y)**2) * fps
                        speeds[track_id] = speed
                    prev_centroids[track_id] = (cx, cy)

                    # Draw bounding box and info
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    label = f"{class_name} ID:{track_id} {speeds.get(track_id, 0):.1f}px/s"
                    cv2.putText(frame, label, (x1, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Update heatmap
                    if heatmap is None:
                        heatmap = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.float32)
                    heatmap[cy, cx] += 1

                    # Alert if needed
                    if class_name in alert_objects:
                        cv2.putText(frame, f"ALERT: {class_name.upper()}", (10, 50),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)

                    # Log data
                    object_stats.append({
                        "Frame": frame_idx,
                        "Time": round(frame_idx / fps, 2),
                        "Object": class_name,
                        "TrackID": track_id,
                        "Speed": speeds.get(track_id, 0),
                        "Position": (cx, cy)
                    })

            # Apply heatmap overlay
            frame = draw_heatmap(frame)

            # Display frame info
            cv2.putText(frame, f"Frame: {frame_idx}", (10, height - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

            out.write(frame)
            if display and frame_idx % 10 == 0:  # Display every 10 frames for performance
                cv2_imshow(frame)

    cap.release()
    out.release()

    print(f"\nOutput saved to: {output_path}")
    print("Total object counts:")
    for obj, count in total_counts.items():
        print(f"  {obj}: {count}")

    df = pd.DataFrame(object_stats)
    df.to_csv("object_tracking_stats.csv", index=False)
    print("Data saved to object_tracking_stats.csv")

# ▶️ Main execution
if __name__ == "__main__":
    video_path = "your_video.mp4"  # Change this to your video path

    process_video(
        video_path=video_path,
        output_path="output_tracked.mp4",
        display=True,
        count_interval_seconds=60
    )

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import time
from tqdm import tqdm
import csv

# For Colab use only (if running on Colab):
from google.colab.patches import cv2_imshow

def count_and_save_objects(video_path, output_path='output_video.mp4',
                           model_type='yolov8n.pt', conf_threshold=0.5,
                           target_classes=None, count_interval_seconds=60,
                           display=True, use_webcam=False):

    cap = cv2.VideoCapture(0) if use_webcam else cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Could not open video.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 1000
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    model = YOLO(model_type)
    _ = model(np.zeros((height, width, 3), dtype=np.uint8))  # warmup

    # Initialize object tracking
    object_stats = []
    total_counts = defaultdict(int)  # To store total counts of each object
    frame_counts = defaultdict(int)  # Per-frame counts within the interval
    object_lifetimes = defaultdict(int)
    trails = defaultdict(lambda: deque(maxlen=30))
    seen_objects = set()

    print("🔍 Processing...")
    frame_idx = 0
    start_time = time.time()
    frame_time_accumulated = 0
    last_count_time = time.time()

    try:
        with tqdm(total=frame_count_total, desc="🔁 Frames") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                frame_idx += 1
                pbar.update(1)

                frame_time_accumulated += 1 / fps
                if frame_time_accumulated < count_interval_seconds:
                    # Count objects continuously as the video is playing
                    detections = []
                    results = model(frame, conf=conf_threshold)
                    for r in results:
                        for box in r.boxes:
                            class_idx = int(box.cls)
                            class_name = r.names.get(class_idx, f"class_{class_idx}")
                            if target_classes and class_name not in target_classes:
                                continue
                            xyxy = box.xyxy[0].cpu().numpy().astype(int)
                            conf = float(box.conf[0])
                            detections.append([*xyxy, conf])  # Store bbox and confidence
                            frame_counts[class_name] += 1  # Update counts for this frame
                            label = f"{class_name} {conf:.2f}"
                            cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
                            cv2.putText(frame, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Display counts after each interval
                    if frame_time_accumulated >= count_interval_seconds:
                        print(f"\n📊 Total counts after {count_interval_seconds} seconds:")
                        for name, count in total_counts.items():
                            print(f"{name}: {count}")

                        # Reset frame counts for the next interval
                        total_counts.update(frame_counts)
                        frame_counts.clear()
                        frame_time_accumulated = 0  # Reset after displaying counts

                # Write and display the output
                out.write(frame)
                if display:
                    cv2_imshow(frame)  # If you're using Colab, otherwise use cv2.imshow locally

                # Store per-frame stats after interval
                timestamp = round(frame_idx / fps, 2)
                for name, count in frame_counts.items():
                    object_stats.append({
                        'Frame': frame_idx,
                        'Time': timestamp,
                        'Object': name,
                        'Count': count
                    })

    finally:
        cap.release()
        out.release()

        with open("object_detection_stats.csv", "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=['Frame', 'Time', 'Object', 'Count'])
            writer.writeheader()
            writer.writerows(object_stats)

        print(f"\n🎥 Output video saved to: {output_path}")
        print("🧾 Stats saved to object_detection_stats.csv")
        print(f"🕒 Elapsed time: {round(time.time() - start_time, 2)}s")
        print("\n📊 FINAL TOTAL OBJECT COUNTS:")
        for name, count in total_counts.items():
            print(f"{name}: {count}")
        print("\n📍 OBJECT LIFETIMES (frames):")
        for obj_id, duration in object_lifetimes.items():
            print(f"ID {obj_id}: {duration}")


# 👇 Run this part after uploading your video to Colab
video_path = 'DaBaby - _BOOK IT_ (Official Music Video)(360P).mp4'

count_and_save_objects(
    video_path=video_path,
    output_path='output_video.mp4',
    model_type='yolov8n.pt',
    conf_threshold=0.5,
    target_classes=None,  # or ['person'] etc.
    count_interval_seconds=60,  # Count objects every 60 seconds
    display=True,  # Set to True for live video display
    use_webcam=False  # Set to True if using a webcam
)